# Quick Start - Interactive Sankey Library

Get started with interactive Sankey diagrams in just a few lines of code!

In [1]:
# Setup
import sys
sys.path.insert(0, '../src')

import pandas as pd
import numpy as np
from sankey_interactive import SankeyDiagram

## Example 1: Simple Flow Diagram

In [2]:
# Create simple data
data = pd.DataFrame({
    'source': ['A', 'A', 'B', 'B', 'C'],
    'target': ['D', 'E', 'D', 'E', 'E'],
    'value': [10, 20, 15, 5, 25]
})

# Create and display
diagram = SankeyDiagram(data)
diagram.render('source', 'target', 'value').show()

## Example 2: Time Series with Animation

In [ ]:
# Create time series data
np.random.seed(42)
dates = pd.date_range('2024-01', periods=6, freq='ME')
sources = ['Solar', 'Wind', 'Coal']
targets = ['Residential', 'Commercial']

data = []
for date in dates:
    for source in sources:
        for target in targets:
            data.append({
                'date': date,
                'source': source,
                'target': target,
                'value': np.random.randint(50, 150)
            })

df = pd.DataFrame(data)

# Create animated diagram with smooth transitions
diagram = SankeyDiagram(df, time_column='date')
diagram.render('source', 'target', 'value', 
               title="Energy Flows Over Time (Click Play!)",
               show_timeline=True,
               animation_duration=1200,  # Slower, smoother transitions (in ms)
               transition_easing='cubic-in-out').show()  # Smooth easing

### Animation Speed Options

You can control the transition speed and smoothness:
- `animation_duration`: Speed in milliseconds (300=fast, 800=normal, 1500=slow)
- `transition_easing`: Smoothness style ('linear', 'cubic-in-out', 'elastic', etc.)

In [ ]:
# Example: Fast animation
diagram_fast = SankeyDiagram(df, time_column='date')
fig_fast = diagram_fast.render('source', 'target', 'value', 
                                title="⚡ Fast Animation (300ms)",
                                show_timeline=True,
                                animation_duration=300,  # Fast!
                                transition_easing='linear')
fig_fast.show()

# Example: Very smooth, slow animation
diagram_slow = SankeyDiagram(df, time_column='date')
fig_slow = diagram_slow.render('source', 'target', 'value', 
                                title="🐢 Smooth & Slow Animation (1500ms)",
                                show_timeline=True,
                                animation_duration=1500,  # Slow and smooth
                                transition_easing='cubic-in-out')
fig_slow.show()

## Example 3: With Filters and Histogram Coloring

In [ ]:
# Add filters
diagram = SankeyDiagram(df, time_column='date')
diagram.add_filter('high_value', lambda d: d[d['value'] > 80])

# Render with histogram coloring and smooth animation
diagram.render('source', 'target', 'value',
               title="High-Value Flows with Color Gradient",
               show_histogram=True,
               show_timeline=True,
               animation_duration=1000,  # Smooth 1-second transitions
               transition_easing='cubic-in-out').show()

## That's it! 🎉

You now have interactive Sankey diagrams with:
- ⏱️ Timeline animation with adjustable speed
- 🌈 Value-based coloring
- 🔍 Dynamic filters
- 🎮 Interactive controls
- ✨ Smooth transitions with customizable easing

### Animation Speed Tips:
- **Fast (300ms)**: Quick transitions, good for presentations
- **Normal (800ms)**: Balanced speed and smoothness (default)
- **Slow (1200-1500ms)**: Very smooth, easy to follow changes

Check out `interactive_demos.ipynb` for more advanced examples!